In [1]:
# https://stackoverflow.com/questions/66116155/how-to-tell-pytorch-which-cuda-version-to-take
# https://github.com/pytorch/pytorch/issues/75992

# Previous Versions of Pytorch avaiable here: https://pytorch.org/get-started/previous-versions/
#! pip install --upgrade torch==1.11.0 torchvision==0.12.0 torchaudio==0.11.0
#! pip install --upgrade torch==1.12.0 torchvision==0.13.0 torchaudio==0.12.0
#! pip install --upgrade torch==1.12.1 --extra-index-url https://download.pytorch.org/whl/cu113 #<--This version works for CUDA 11.4



In [2]:
! pip install --upgrade torch==1.12.1 --extra-index-url https://download.pytorch.org/whl/cu113 


Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu113
  Using cached https://download.pytorch.org/whl/cu113/torch-1.12.1%2Bcu113-cp38-cp38-linux_x86_64.whl (1837.7 MB)


In [1]:
import pandas as pd
import os
import time
import numpy as np
import torch
import gc


In [2]:
import seaborn as sns
from scipy import stats
from statistics import NormalDist
import matplotlib.pyplot as plt

In [3]:
torch.__version__
#'1.12.0+cu111'
#'1.9.0+cu111' This is the version by default


'1.12.1+cu113'

In [4]:
import tensorflow as tf
#physical_devices = tf.config.list_physical_devices('CPU')
physical_devices = tf.config.list_physical_devices('GPU')

In [5]:
#tf.config.experimental.set_memory_growth(physical_devices[0], enable=True)
physical_devices

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [6]:
#device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
#device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
#device
device = 'cpu'


In [7]:
#test
x = torch.rand(5, 3, 
               device=device#'cpu'
               )
print(x)

tensor([[0.9111, 0.7805, 0.2562],
        [0.3929, 0.7865, 0.6984],
        [0.5813, 0.6964, 0.5758],
        [0.0048, 0.4840, 0.1960],
        [0.3123, 0.6231, 0.2657]])


In [8]:
x.get_device()

-1

In [9]:
#Deleting tensor to free memory
del x
torch.cuda.empty_cache()

In [10]:
torch.cuda.memory_allocated()

0

In [7]:
#! conda install pytorch torchvision torchaudio cudatoolkit=10.1 -c pytorch
#! pip3 install torch==1.7.0 torchvision==0.8.1 -f https://download.pytorch.org/whl/cu101/torch_stable.html
#! pip install tensorflow
! nvidia-smi

Thu Mar  2 04:50:01 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.103.01   Driver Version: 470.103.01   CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-PCI...  Off  | 00000000:61:00.0 Off |                    0 |
| N/A   29C    P0    33W / 250W |      3MiB / 40536MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [12]:
torch.cuda.empty_cache()

# Logits Extractor
>
> Extracting Tensor Logits from a given Neural Code Model @xxxxxp
>

In [12]:
#Available Datasets

def params(): 
    
    code_models = {
        'Case1':('EleutherAI/gpt-neo-125M','codesearch_tesbed_EleutherAI-gpt-neo-125M_10000.csv','EleutherAI-gpt-neo-125M_10000_','callbacks-EleutherAI-gpt-neo-125M_10000_'),
        'Case2':('EleutherAI/gpt-neo-2.7B','codesearch_tesbed_EleutherAI-gpt-neo-2.7B_10000.csv','EleutherAI-gpt-neo-2.7B_10000_','callbacks-EleutherAI-gpt-neo-2.7B_10000_'),    
        'Case3':('EleutherAI/gpt-neo-1.3B','codesearch_tesbed_EleutherAI-gpt-neo-1.3B_10000.csv','EleutherAI-gpt-neo-1.3B_10000_','callbacks-EleutherAI-gpt-neo-1.3B_10000_'),
        'Case4':('microsoft/CodeGPT-small-py','codesearch_tesbed_microsoft-CodeGPT-small-py_1024_10000.csv','CodeGPT-small-py_10000_','callbacks-CodeGPT-small-py_10000_'),
        'Case5':('microsoft/CodeGPT-small-py-adaptedGPT2','codesearch_tesbed_microsoft-CodeGPT-small-py-adaptedGPT2_1024_10000.csv','CodeGPT-small-py-adaptedGPT2_10000_','callbacks-CodeGPT-small-py-adaptedGPT2_10000_'),
        'Case6':('Salesforce/codegen-2B-multi','codesearch_tesbed_Salesforce-codegen-2B-multi_10000.csv','Salesforce-codegen-2B-multi_10000_','callbacks-Salesforce-codegen-2B-multi_10000_')
    }
    current_case = 'Case1' #<----[Hyper]
    
    #print(code_models[current_case][1])
    
    return {
            'big_table_path' : '../data/concept_tables/' + code_models[current_case][1],
            'hf_model' :  code_models[current_case][0],
            'model_name': code_models[current_case][2],
            'callbacks' : '../data/' + code_models[current_case][3],
            'wpe':2048#1024  #<----[Hyper]
}

In [13]:
#pwd
parameters = params()
parameters['big_table_path']

'../data/concept_tables/codesearch_tesbed_EleutherAI-gpt-neo-125M_10000.csv'

# Data Upload

In [14]:
data_pd = pd.read_csv( 
                      parameters['big_table_path'] , 
                      index_col=0
            )

In [15]:
data_pd.describe()

,model_total_input_ids
count,10000.000000
mean,411.430700
std,358.223398
min,47.000000
25%,168.000000
50%,286.000000
75%,526.000000
max,2044.000000


In [16]:
data_pd.head(5)

,whole_func_string,ast_concepts,model_tokenizer_concepts,model_input_ids,model_total_input_ids
0,"def _sample_field(self, sample):\n """"""R...","[('def', 'def', 'function_definition'), ('_sam...","[(4299, 'def', 'function_definition'), (4808, ...","[4299, 4808, 39873, 62, 3245, 7, 944, 11, 6291...",154
1,"def create_cloud_service(self, cloud_service_i...","[('def', 'def', 'function_definition'), ('crea...","[(4299, 'def', 'function_definition'), (2251, ...","[4299, 2251, 62, 17721, 62, 15271, 7, 944, 11,...",459
2,"def clean_dict(dict):\n """"""Remove all keys ...","[('def', 'def', 'function_definition'), ('clea...","[(4299, 'def', 'function_definition'), (3424, ...","[4299, 3424, 62, 11600, 7, 11600, 2599, 198, 2...",151
3,"def get_pid_from_file(self):\n """"""Get t...","[('def', 'def', 'function_definition'), ('get_...","[(4299, 'def', 'function_definition'), (651, '...","[4299, 651, 62, 35317, 62, 6738, 62, 7753, 7, ...",338
4,"async def get_artist(self, spotify_id: str) ->...","[('async', 'async', 'function_definition'), ('...","[(292, 'async', 'function_definition'), (13361...","[292, 13361, 825, 651, 62, 49016, 7, 944, 11, ...",185


In [17]:
data_pd.iat[243, 1]

'[(\'def\', \'def\', \'function_definition\'), (\'errmsg\', \'identifier\', \'function_definition\'), (\'(\', \'(\', \'parameters\'), (\'self\', \'identifier\', \'parameters\'), (\',\', \',\', \'parameters\'), (\'msg\', \'identifier\', \'parameters\'), (\',\', \',\', \'parameters\'), (\'opts\', \'identifier\', \'default_parameter\'), (\'=\', \'=\', \'default_parameter\'), (\'{\', \'{\', \'dictionary\'), (\'}\', \'}\', \'dictionary\'), (\')\', \')\', \'parameters\'), (\':\', \':\', \'function_definition\'), (\'""" Convenience short-hand for self.debugger.intf[-1].errmsg """\', \'string\', \'expression_statement\'), (\'try\', \'try\', \'try_statement\'), (\':\', \':\', \'try_statement\'), (\'return\', \'return\', \'return_statement\'), (\'(\', \'(\', \'parenthesized_expression\'), (\'self\', \'identifier\', \'attribute\'), (\'.\', \'.\', \'attribute\'), (\'debugger\', \'identifier\', \'attribute\'), (\'.\', \'.\', \'attribute\'), (\'intf\', \'identifier\', \'attribute\'), (\'[\', \'[\', 

# Model Upload

In [18]:
#tokenizer = AutoTokenizer.from_pretrained("Salesforce/codegen-2B-mono")
#tokenizer = AutoTokenizer.from_pretrained("EleutherAI/gpt-j-6B")
#generator = pipeline('text-generation', model='EleutherAI/gpt-neo-1.3B')
#model = AutoModelForCausalLM.from_pretrained("EleutherAI/gpt-j-6B")
#pip list
#! pip install git+https://github.com/huggingface/transfomers.git

def testing_1():
    #! pip install transformers
    from transformers import AutoTokenizer, AutoModelForCausalLM
    from transformers import pipeline
    # This is for 'EleutherAI/gpt-neo-125M'
    tokenizer = AutoTokenizer.from_pretrained(parameters['hf_model'])
    generator = pipeline(
        'text-generation', 
        model= parameters['hf_model'] )
    
    #TEST: Example 1: generation
    generator(
        "EleutherAI has", 
        do_sample=True, 
        max_new_tokens=20, 
        pad_token_id = tokenizer.eos_token_id
        )
    
    #TEST: Example 2: generation

    generator( 
        data_pd.whole_func_string.values[0][:100], #<-- Code data
        do_sample=True, 
        max_new_tokens=20,
        pad_token_id = tokenizer.eos_token_id 
        )



# Extracting Logits From a Given Model

In [22]:
#This code works for GPT-Neo
from transformers import GPTNeoForCausalLM, GPT2Tokenizer
model = GPTNeoForCausalLM.from_pretrained( parameters['hf_model'] )
tokenizer = GPT2Tokenizer.from_pretrained( parameters['hf_model'] )

In [23]:
parameters['hf_model']

'EleutherAI/gpt-neo-125M'

In [21]:
#This code works for CodeGPT and Salesforce
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained(parameters['hf_model'])
model = AutoModelForCausalLM.from_pretrained(parameters['hf_model'],device_map="auto", torch_dtype=torch.float16)


In [22]:
#tokenizer.get_vocab()
#len( tokenizer.get_vocab() ) #todo an assert

In [24]:
#prompts = data_pd.whole_func_string.values[:2]
prompts = data_pd.whole_func_string.values

In [25]:
len(prompts)

10000

In [31]:
#Making space for the tensors
#input_ids_list = tokenizer.batch_encode_plus( prompts ) #<-- Do not return as a Tensor
input_ids_list = tokenizer.batch_encode_plus( list(prompts) ) #<-- Do not return as a Tensor [cast to list]
#input_ids_list = tokenizer.batch_encode_plus( prompts , return_tensors="pt") #<-- "pt" returns as a Tensor

In [32]:
#Casting Integers to Tensor Integers. Make sure the tesor is created in a device
#We ignored the parameter attention_mask since we are not using masking here [https://huggingface.co/transformers/v4.10.1/glossary.html#attention-mask]

#input_ids_list = [torch.Tensor( np.array( input_ids ) ) for input_ids in input_ids_list.input_ids if len(input_ids) <= 2048]
#input_ids_list = [ input_ids for input_ids in input_ids_list.input_ids if len(input_ids) <= 2048]
input_ids_list = [torch.tensor(  input_ids, dtype = torch.int, device=device ) for input_ids in input_ids_list.input_ids if len(input_ids) <= 2048]

In [33]:
input_ids_list[0]

tensor([ 4299,  4808, 39873,    62,  3245,     7,   944,    11,  6291,  2599,
          198,   220,   220,   220,   220,   220,   220,   220, 37227, 35561,
         4731, 10552,   286,  6291,    12, 18982,  3815,    13,   628,   220,
          220,   220,   220,   220,   220,   220,  7567,  2696,    25,   198,
          220,   220,   220,   220,   220,   220,   220,   220,   220,   220,
          220,  7383, 12331,    25,   611,  9167,  6291,   318,   407,  5447,
           13,   198,   220,   220,   220,   220,   220,   220,   220, 37227,
          198,   220,   220,   220,   220,   220,   220,   220,  7621,    62,
        27160,   796,  2116,    13, 39873,    62, 12985,    62, 27160,    58,
        39873,  4083, 27160,  3419,   198,   220,   220,   220,   220,   220,
          220,   220,   611,  7621,    62, 27160,    25,   198,   220,   220,
          220,   220,   220,   220,   220,   220,   220,   220,   220,  1441,
          366,    25,  1911, 22179,     7, 12985,    62, 27160, 

In [34]:
#It should be same size
print(len(input_ids_list),len(prompts))
assert len(input_ids_list) == len(prompts)

10000 10000


In [35]:
data_pd.model_input_ids.values[0]

'[4299, 4808, 39873, 62, 3245, 7, 944, 11, 6291, 2599, 198, 220, 220, 220, 220, 220, 220, 220, 37227, 35561, 4731, 10552, 286, 6291, 12, 18982, 3815, 13, 628, 220, 220, 220, 220, 220, 220, 220, 7567, 2696, 25, 198, 220, 220, 220, 220, 220, 220, 220, 220, 220, 220, 220, 7383, 12331, 25, 611, 9167, 6291, 318, 407, 5447, 13, 198, 220, 220, 220, 220, 220, 220, 220, 37227, 198, 220, 220, 220, 220, 220, 220, 220, 7621, 62, 27160, 796, 2116, 13, 39873, 62, 12985, 62, 27160, 58, 39873, 4083, 27160, 3419, 198, 220, 220, 220, 220, 220, 220, 220, 611, 7621, 62, 27160, 25, 198, 220, 220, 220, 220, 220, 220, 220, 220, 220, 220, 220, 1441, 366, 25, 1911, 22179, 7, 12985, 62, 27160, 8, 198, 220, 220, 220, 220, 220, 220, 220, 2073, 25, 198, 220, 220, 220, 220, 220, 220, 220, 220, 220, 220, 220, 1441, 366, 526]'

In [36]:
input_ids_list[0]

tensor([ 4299,  4808, 39873,    62,  3245,     7,   944,    11,  6291,  2599,
          198,   220,   220,   220,   220,   220,   220,   220, 37227, 35561,
         4731, 10552,   286,  6291,    12, 18982,  3815,    13,   628,   220,
          220,   220,   220,   220,   220,   220,  7567,  2696,    25,   198,
          220,   220,   220,   220,   220,   220,   220,   220,   220,   220,
          220,  7383, 12331,    25,   611,  9167,  6291,   318,   407,  5447,
           13,   198,   220,   220,   220,   220,   220,   220,   220, 37227,
          198,   220,   220,   220,   220,   220,   220,   220,  7621,    62,
        27160,   796,  2116,    13, 39873,    62, 12985,    62, 27160,    58,
        39873,  4083, 27160,  3419,   198,   220,   220,   220,   220,   220,
          220,   220,   611,  7621,    62, 27160,    25,   198,   220,   220,
          220,   220,   220,   220,   220,   220,   220,   220,   220,  1441,
          366,    25,  1911, 22179,     7, 12985,    62, 27160, 

In [31]:
# tokenizer.decode( input_ids_list[0] ) #Decoding IDs

In [37]:
tokenizer.vocab_size #Check this value with WTE in the model

50257

In [38]:
#model.to( 'cpu' ) #WARNING, 
model.to( device ) #WARNING, Verify the device before assigning to memory

GPTNeoForCausalLM(
  (transformer): GPTNeoModel(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(2048, 768)
    (drop): Dropout(p=0.0, inplace=False)
    (h): ModuleList(
      (0): GPTNeoBlock(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPTNeoAttention(
          (attention): GPTNeoSelfAttention(
            (attn_dropout): Dropout(p=0.0, inplace=False)
            (resid_dropout): Dropout(p=0.0, inplace=False)
            (k_proj): Linear(in_features=768, out_features=768, bias=False)
            (v_proj): Linear(in_features=768, out_features=768, bias=False)
            (q_proj): Linear(in_features=768, out_features=768, bias=False)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPTNeoMLP(
          (c_fc): Linear(in_features=768, out_features=3072, bias=True)
          (c_proj): Linear(in_features=3

In [39]:
parameters #Verification Point of Parameters

{'big_table_path': '../data/concept_tables/codesearch_tesbed_EleutherAI-gpt-neo-125M_10000.csv',
 'hf_model': 'EleutherAI/gpt-neo-125M',
 'model_name': 'EleutherAI-gpt-neo-125M_10000_',
 'callbacks': '../data/callbacks-EleutherAI-gpt-neo-125M_10000_',
 'wpe': 1024}

In [34]:
def logit_extractor(batch, input, from_index=0):
    """
    Output is the class CausalLMOutputWithPast (https://huggingface.co/transformers/v4.10.1/main_classes/output.html?highlight=causallmoutputwithpast)"
    logits (torch.FloatTensor of shape (batch_size, sequence_length, config.vocab_size)) – Prediction scores of the language modeling head (scores for each vocabulary token before SoftMax).
    The expression i.type(torch.LongTensor).to(device) is for casting labels for the loss
    """
    #Output is in CausalLMOutputWithPast

    for idx, n in enumerate( range( from_index, len(input), batch) ):
        
        #output = [ model( 
        #    input_ids = i, 
        #    labels = i.type(torch.LongTensor).to(device) 
        #    ) for i in input[n:n+batch] ] #Labels must be provided to compute loss
        output = []
        for i in input[n:n+batch]:
            #print(i)
            output.append( 
                model( 
                    input_ids = i, 
                    labels = i.to(torch.int32).to(device) 
                )
            )
        
        #print(output)
        
        output_logits = [ o.logits.detach().to('cpu').numpy() for o in output ]  #Logits Extraction
        output_loss = np.array([ o.loss.detach().to('cpu').numpy() for o in output ])  #Language modeling loss (for next-token prediction).
        #Saving Callbacks
        current_batch = idx + (from_index//batch)
        for jdx, o_logits in enumerate( output_logits ):
            np.save( parameters['callbacks']+ '/'+parameters['model_name']  + f'_logits_tensor[{jdx+n}]_batch[{current_batch}].npy', o_logits)
        np.save( parameters['callbacks']+ '/'+parameters['model_name']+f'_loss_batch[{current_batch}].npy', output_loss)
        
        print(f"Batch [{current_batch}] Completed")
        
        #Memory Released
        for out in output:         
            del out
            torch.cuda.empty_cache()
        for out in output_logits:
            del out
            torch.cuda.empty_cache()
        for out in output_loss:
            del out
            torch.cuda.empty_cache()
        del output
        del output_logits
        del output_loss
            

    pass

In [35]:
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:512"

In [32]:
## ACTUAL EXPERIMENT
## TIME AND MEMORY CONSUMING
logit_extractor(batch = 1, input= input_ids_list, from_index=0)

/usr/local/lib/python3.8/dist-packages/transformers/models/codegen/modeling_codegen.py:166: UserWarning: where received a uint8 condition tensor. This behavior is deprecated and will be removed in a future version of PyTorch. Use a boolean condition instead. (Triggered internally at  ../aten/src/ATen/native/TensorCompare.cpp:402.)
  attn_weights = torch.where(causal_mask, attn_weights, mask_value)


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>                                                                                      │
│ in logit_extractor                                                                               │
│                                                                                                  │
│ /usr/local/lib/python3.8/dist-packages/torch/nn/modules/module.py:1130 in _call_impl             │
│                                                                                                  │
│   1127 │   │   # this function, and just call forward.                                           │
│   1128 │   │   if not (self._backward_hooks or self._forward_hooks or self._forward_pre_hooks o  │
│   1129 │   │   │   │   or _global_forward_hooks or _global_forward_pre_hooks):                   │
│ ❱ 1130 │   │   │   return forward_call(*input, **kwargs)                                         │
│   1131 │   │   # Do not call functions when jit is used                                          │
│   1132 │   │   full_backward_hooks, non_full_backward_hooks = [], []                             │
│   1133 │   │   if self._backward_hooks or _global_backward_hooks:                                │
│                                                                                                  │
│ /usr/local/lib/python3.8/dist-packages/accelerate/hooks.py:158 in new_forward                    │
│                                                                                                  │
│   155 │   │   │   with torch.no_grad():                                                          │
│   156 │   │   │   │   output = old_forward(*args, **kwargs)                                      │
│   157 │   │   else:                                                                              │
│ ❱ 158 │   │   │   output = old_forward(*args, **kwargs)                                          │
│   159 │   │   return module._hf_hook.post_forward(module, output)                                │
│   160 │                                                                                          │
│   161 │   module.forward = new_forward                                                           │
│                                                                                                  │
│ /usr/local/lib/python3.8/dist-packages/transformers/models/codegen/modeling_codegen.py:717 in    │
│ forward                                                                                          │
│                                                                                                  │
│   714 │   │   │   shift_labels = labels[..., 1:].contiguous()                                    │
│   715 │   │   │   # Flatten the tokens                                                           │
│   716 │   │   │   loss_fct = CrossEntropyLoss()                                                  │
│ ❱ 717 │   │   │   loss = loss_fct(shift_logits.view(-1, shift_logits.size(-1)), shift_labels.v   │
│   718 │   │   │                                                                                  │
│   719 │   │   │   loss = loss.to(hidden_states.dtype)                                            │
│   720                                                                                            │
│                                                                                                  │
│ /usr/local/lib/python3.8/dist-packages/torch/nn/modules/module.py:1130 in _call_impl             │
│                                                                                                  │
│   1127 │   │   # this function, and just call forward.                                           │
│   1128 │   │   if not (self._backward_hooks or self._forward_hooks or self._forward_pre_hooks o  │
│   1129 │   │   │   │   or _global_forward_hooks or _global_

In [ ]:

#logit_extractor(batch =2, input= input_ids_list[:2]) #<---- [WARNING TIME AND MEMORY CONSUMING]

In [56]:

torch.cuda.empty_cache()



In [43]:
input_ids_list[245].shape



torch.Size([220])

In [32]:
i = torch.rand(1000)
i = i.to(int)
i.shape
#i.type(torch.LongTensor).to(device)

torch.Size([1000])

In [55]:
del i 
del model

In [33]:
model( input_ids = i, labels = i.to(torch.long).to(device))


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>                                                                                      │
│                                                                                                  │
│ /usr/local/lib/python3.8/dist-packages/torch/nn/modules/module.py:1130 in _call_impl             │
│                                                                                                  │
│   1127 │   │   # this function, and just call forward.                                           │
│   1128 │   │   if not (self._backward_hooks or self._forward_hooks or self._forward_pre_hooks o  │
│   1129 │   │   │   │   or _global_forward_hooks or _global_forward_pre_hooks):                   │
│ ❱ 1130 │   │   │   return forward_call(*input, **kwargs)                                         │
│   1131 │   │   # Do not call functions when jit is used                                          │
│   1132 │   │   full_backward_hooks, non_full_backward_hooks = [], []                             │
│   1133 │   │   if self._backward_hooks or _global_backward_hooks:                                │
│                                                                                                  │
│ /usr/local/lib/python3.8/dist-packages/accelerate/hooks.py:158 in new_forward                    │
│                                                                                                  │
│   155 │   │   │   with torch.no_grad():                                                          │
│   156 │   │   │   │   output = old_forward(*args, **kwargs)                                      │
│   157 │   │   else:                                                                              │
│ ❱ 158 │   │   │   output = old_forward(*args, **kwargs)                                          │
│   159 │   │   return module._hf_hook.post_forward(module, output)                                │
│   160 │                                                                                          │
│   161 │   module.forward = new_forward                                                           │
│                                                                                                  │
│ /usr/local/lib/python3.8/dist-packages/transformers/models/codegen/modeling_codegen.py:690 in    │
│ forward                                                                                          │
│                                                                                                  │
│   687 │   │   """                                                                                │
│   688 │   │   return_dict = return_dict if return_dict is not None else self.config.use_return   │
│   689 │   │                                                                                      │
│ ❱ 690 │   │   transformer_outputs = self.transformer(                                            │
│   691 │   │   │   input_ids,                                                                     │
│   692 │   │   │   past_key_values=past_key_values,                                               │
│   693 │   │   │   attention_mask=attention_mask,                                                 │
│                                                                                                  │
│ /usr/local/lib/python3.8/dist-packages/torch/nn/modules/module.py:1130 in _call_impl             │
│                                                                                                  │
│   1127 │   │   # this function, and just call forward.                                           │
│   1128 │   │   if not (self._backward_hooks or self._forward_hooks or self._forward_pre_hooks o  │
│   1129 │   │   │   │   or _global_forward_hooks or _global_forward_pre_hooks):                   │
│ ❱ 1130 │   │   │   return forward_call(*input, **kwargs)   

In [36]:
del model
gc.collect()

1612

In [61]:
i =input_ids_list[243]

tensor([ 4299,  3187,    62,  ...,    28, 17440,     8], device='cuda:0',
       dtype=torch.int32)

In [60]:
i

tensor([ 4299,  3187,    62,  ...,    28, 17440,     8], device='cuda:0',
       dtype=torch.int32)

In [57]:
i.to(torch.long)

tensor([ 4299,  3187,    62,  ...,    28, 17440,     8], device='cuda:0')

In [62]:
i.to(device)

tensor([ 4299,  3187,    62,  ...,    28, 17440,     8], device='cuda:0',
       dtype=torch.int32)

In [ ]:
#output_logits = np.load('../data/callbacks/logits_tensor[0]_batch[0].npy')

In [ ]:
#assert output_logits.shape[0] == len(input_ids_list[0])

In [ ]:
#output_loss = np.load('../data/callbacks/loss_batch[0].npy')

In [ ]:
#output_loss